<style>
.header {
    background: linear-gradient(90deg, #071124 0%, #0b5261 50%, #2fb5b0 100%);
    color: #ffffff;
    padding: 20px 26px;
    border-radius: 12px;
    text-align: center;
    box-shadow: 0 6px 18px rgba(11, 37, 60, 0.45);
    margin-bottom: 16px;
    font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, "Helvetica Neue", Arial;
}
.header h1 {
    margin: 0;
    font-size: 1.8rem;
    letter-spacing: -0.5px;
}
.header p {
    margin: 6px 0 0;
    color: rgba(255,255,255,0.92);
    font-size: 0.96rem;
    text-align: center;
    display: block;
    max-width: 800px;
    margin-left: auto;
    margin-right: auto;
    background: rgba(255,255,255,0.04);
    padding: 8px 12px;
    border-radius: 10px;
}
.badges {
    margin-top: 10px;
}
.badge {
    display: inline-block;
    background: rgba(255,255,255,0.12);
    color: #e6f7f7;
    padding: 6px 10px;
    border-radius: 999px;
    font-size: 0.78rem;
    margin: 0 6px;
}
.kv {
    margin-top: 12px;
    text-align: left;
    max-width: 900px;
}
.kv dt { font-weight: 600; }
.kv dd { margin: 4px 0 10px 0; color: #333; }

.header h2 {
    font-size: 1.05rem; /* smaller than h1 (1.8rem) */
    margin: 6px 0 0;
    font-weight: 500;
    color: rgba(255,255,255,0.95);
}
</style>

<div class="header">
    <h1>Exotic European option 1</h1>
    <h2>Closed-form derivation - pricing and Monte Carlo simulation</h2>
    <p>Monte Carlo simulation and analytical formulas for exotic option pricing</p>
    <div class="badges">
        <span class="badge">Exotic European Option</span>
        <span class="badge">Analytical Solution</span>
        <span class="badge">Monte Carlo</span>
    </div>
</div>

---

# Overview
- Derive and implement closed-form solutions.
- Perform Monte Carlo simulations to price and verify the correctness of analytical formulas.

# Import packages

In [1]:
import numpy as np
from scipy.stats import norm

# Problem Statement
Consider a European call option with maturity $T$ on a foreign asset $\tilde S$, which pays (in domestic currency) at time $T$
$$
\max[0,X(T)\tilde S(T)-X(0)\tilde S(T)]
$$
Assume that the exchange rate $X$ (in units of domestic currency per unit of foreign currency) follows the dynamics (under the domestic risk-neutral measure) of
$$
dX(t) = X(t)((r-\tilde r)dt+\sigma^\top dW(t))
$$
with $W(t)$ $d$-dimensional standard Brownian motions under domestic risk-neutral measure, $r$ and $\tilde r$ are domestic and foreign risk-free rate, which are scalar constants, and $\sigma$ is exchange rate's volatility $d$-dimensional, which is also a constant vector.

Also, $\tilde S$ follows the dynamics (under the foreign risk-neutral measure) of
$$
d\tilde S(t) = \tilde S(t)(\tilde rdt+\xi^\top d\tilde W(t))
$$
with $\tilde W(t)$ $d$-dimensional standard Brownian motions under foreign risk-neutral measure, and $\xi$ is foreign asset's volatility $d$-dimensional constant vector.

# Analytical Derivation

Price of the option can be derived using risk-neutral valuation. The price at time t of the option is given by the discounted expected payoff under the domestic risk-neutral measure:
$$V(t) = e^{-r(T-t)} \mathbb{E}^{\mathbb{Q}}[\max(0, X(T)\tilde S(T) - X(0)\tilde S(T))|\mathcal{F}_t]$$
First, we need to get the solutions for $\tilde S(T)$ and $X(T)$. Given the SDEs, we can solve them with the following results:
$$\tilde S(T) = \tilde S(t) \exp\left((\tilde r - \frac{1}{2} \|\xi\|^2)(T-t) + \xi^\top (\tilde W(T)-\tilde W(t))\right)$$
$$X(T) = X(t) \exp\left((r - \tilde r - \frac{1}{2} \|\sigma\|^2)(T-t) + \sigma^\top (W(T)-W(t))\right)$$
The relationship between the two Brownian motions $W(t)$ and $\tilde W(t)$ can be found via the relation between the two measures. The Radon-Nikodym derivative between the domestic and foreign risk-neutral measures is given by:

$$\frac{d\mathbb{\tilde Q}}{d\mathbb{Q}}\Big|\mathcal{F}_{T|t}=\frac{X(T)e^{-\tilde r(T-T)}}{e^{-r(T-T)}}\frac{e^{-r(T-t)}}{X(t)e^{-\tilde r(T-t)}}=\exp\left( -\frac{1}{2}\|\sigma\|^2(T-t)+\sigma^\top (W(T)-W(t)) \right)$$

This is a valid Radon-Nikodym derivative as it is an exponential martingale under the domestic risk-neutral measure. Hence, under the Girsanov theorem, we have:
$$
d\tilde W(t)=dW(t)-\sigma dt
$$
Hence, we have:
$$
\tilde W(T)-\tilde W(t)=W(T)-W(t)-\sigma (T-t)
$$
which leads to the following:
$$\tilde S(T) = \tilde S(t) \exp\left((\tilde r - \frac{1}{2} \|\xi\|^2-\xi^\top \sigma)(T-t) + \xi^\top (W(T)-W(t))\right)$$

Then, we can rewrite the option valuation formula as:
$$
\begin{aligned}
V(t) &= e^{-r(T-t)} \mathbb{E}^{\mathbb{Q}}[ (X(T)\tilde S(T) - X(0)\tilde S(T))\mathbb{I}_{X(T)\tilde S(T) - X(0)\tilde S(T) > 0}|\mathcal{F}_t]\\
&=e^{-r(T-t)} \left(\mathbb{E}^{\mathbb{Q}}[ X(T)\tilde S(T)\mathbb{I}_{X(T) > X(0)}|\mathcal{F}_t] -\mathbb{E}^{\mathbb{Q}}[  X(0)\tilde S(T)\mathbb{I}_{X(T) > X(0)}|\mathcal{F}_t]\right)\\
\end{aligned}
$$
For the first expectation term, we can use the change of measure technique to simplify the calculation. Define a new probability measure $\mathbb{P}_{\tilde S}$ that accepts the Radon-Nikodym derivative:
$$\frac{d\mathbb{P}_{\tilde S}}{d\mathbb{Q}} = \frac{\tilde S(T)X(T)}{e^{-r(T-T)}} \frac{e^{-r(T-t)}}{\tilde S(t)X(t)}=\exp\left(-\frac{1}{2}\|\sigma+\xi\|^2(T-t)+(\sigma+\xi)^\top (W(T)-W(t))\right)$$
which is a valid Radon-Nikodym derivative as it is an exponential martingale under the domestic risk-neutral measure. Under this new measure, we have:
$$
\begin{aligned}
\mathbb{E}^{\mathbb{Q}}[ X(T)\tilde S(T)\mathbb{I}_{X(T) > X(0)}|\mathcal{F}_t]&=\mathbb{E}^{\tilde S}\left[ X(T)\tilde S(T)\mathbb{I}_{X(T) > X(0)}\frac{1}{\frac{\tilde S(T)X(T)}{e^{-r(T-T)}} \frac{e^{-r(T-t)}}{\tilde S(t)X(t)}}\Bigg|\mathcal{F}_t\right]\\
&=\tilde S(t)X(t)e^{r(T-t)}\mathbb{P}_{\tilde S}(X(T)>X(0)|\mathcal{F}_t)\\
\end{aligned}
$$
According to the Girsanov theorem, under the new measure $\mathbb{P}_{\tilde S}$, the Brownian motion is given by:
$$
d W_{\tilde S}(t)=dW(t)-(\sigma+\xi) dt
$$
$$
\Rightarrow W_{\tilde S}(T)-W_{\tilde S}(t)=W(T)-W(t)-(\sigma+\xi) (T-t)
$$
$$
\Rightarrow X(T) = X(t) \exp\left((r - \tilde r + \xi^\top \sigma + \|\sigma\|^2 - \frac{1}{2} \|\sigma\|^2)(T-t) + \sigma^\top (W_{\tilde S}(T)-W_{\tilde S}(t))\right)
$$
$$
\Rightarrow \ln\left(\frac{X(T)}{X(0)}\right) \sim \mathcal{N}\left(\ln\left(\frac{X(t)}{X(0)}\right)+(r - \tilde r + \xi^\top \sigma + \frac{1}{2} \|\sigma\|^2)(T-t), \|\sigma\|^2(T-t)\right)
$$
Thus, we can calculate the probability term as:
$$
\mathbb{P}_{\tilde S}(X(T)>X(0)|\mathcal{F}_t)=\Phi\left(\frac{\ln\left(\frac{X(t)}{X(0)}\right)+(r - \tilde r + \xi^\top \sigma + \frac{1}{2} \|\sigma\|^2)(T-t)}{\|\sigma\|\sqrt{T-t}}\right)
$$
Similarly, for the second expectation term, we can define another probability measure $\bar{\mathbb{P}}$ that accepts the Radon-Nikodym derivative:
$$\frac{d\bar{\mathbb{P}}}{d\mathbb{Q}} = \frac{\tilde S(T)}{e^{-\tilde r(T-T)}} \frac{e^{-\tilde r(T-t)}}{\tilde S(t)}e^{\xi^\top\sigma(T-t)}=\exp\left(-\frac{1}{2}\|\xi\|^2(T-t)+\xi^\top (W(T)-W(t))\right)$$
which is a valid Radon-Nikodym derivative as it is an exponential martingale under the domestic risk-neutral measure. Under this new measure, we have:
$$
\begin{aligned}
\mathbb{E}^{\mathbb{Q}}[  X(0)\tilde S(T)\mathbb{I}_{X(T) > X(0)}|\mathcal{F}_t]&=\mathbb{E}^{\bar{\mathbb{P}}}\left[  X(0)\tilde S(T)\mathbb{I}_{X(T) > X(0)}\frac{1}{\frac{\tilde S(T)}{e^{-\tilde r(T-T)}} \frac{e^{-\tilde r(T-t)}}{\tilde S(t)}e^{\xi^\top\sigma(T-t)}}\Bigg|\mathcal{F}_t\right]\\
&= X(0)\tilde S(t)e^{(\tilde r-\xi^\top\sigma)(T-t)}\bar{\mathbb{P}}(X(T)>X(0)|\mathcal{F}_t)\\
\end{aligned}
$$
According to the Girsanov theorem, under the new measure $\bar{\mathbb{P}}$, the Brownian motion is given by:
$$
d W_{\bar{\mathbb{P}}}(t)=dW(t)-\xi dt
$$
$$
\Rightarrow W_{\bar{\mathbb{P}}}(T)-W_{\bar{\mathbb{P}}}(t)=W(T)-W(t)-\xi (T-t)
$$
$$
\Rightarrow X(T) = X(t) \exp\left((r - \tilde r + \xi^\top \sigma - \frac{1}{2} \|\sigma\|^2)(T-t) + \sigma^\top (W_{\bar{\mathbb{P}}}(T)-W_{\bar{\mathbb{P}}}(t))\right)
$$
$$
\Rightarrow \ln\left(\frac{X(T)}{X(0)}\right) \sim \mathcal{N}\left(\ln\left(\frac{X(t)}{X(0)}\right)+(r - \tilde r + \xi^\top \sigma - \frac{1}{2} \|\sigma\|^2)(T-t), \|\sigma\|^2(T-t)\right)
$$
Thus, we can calculate the probability term as:
$$
\bar{\mathbb{P}}(X(T)>X(0)|\mathcal{F}_t)=\Phi\left(\frac{\ln\left(\frac{X(t)}{X(0)}\right)+(r - \tilde r + \xi^\top \sigma - \frac{1}{2} \|\sigma\|^2)(T-t)}{\|\sigma\|\sqrt{T-t}}\right)
$$
Furthermore, in order to be useful in practice, we define $\rho$ is the correlation between two dynamics of the foreign asset and the exchange rate, i.e.,
$$\rho=\frac{\xi^\top \sigma}{\|\xi\|\|\sigma\|}$$
By having this correlation term, we don't need to know the full vectors of $\xi$ and $\sigma$, but only their magnitudes and correlation. Combining all the results, we have the final closed-form solution for the option price:
$$
\begin{aligned}
V(t) &= \tilde S(t)X(t)\Phi(d_1) - X(0)\tilde S(t)e^{(\tilde r-r-\rho\|\xi\|\|\sigma\|)(T-t)}\Phi(d_2)\\
d_{1,2}&=\frac{\ln\left(\frac{X(t)}{X(0)}\right)+(r - \tilde r + \rho\|\xi\|\|\sigma\| \pm \frac{1}{2} \|\sigma\|^2)(T-t)}{\|\sigma\|\sqrt{T-t}}\\
\end{aligned}
$$


# Implementation

In [2]:
def Task3Option(S, X, X0, r, r_f, T, t, volS, volX, rho):
    d1 = (
        ((volX**2) / 2 + r - r_f + rho * volS * volX) * (T - t) + np.log(X / X0)
    ) / (volX * np.sqrt(T - t))
    d2 = d1 - volX * np.sqrt(T - t)
    option_price = S * (
        X * norm.cdf(d1)
        - X0 * np.exp((r_f - r - rho * volS * volX) * (T - t)) * norm.cdf(d2)
    )
    return option_price


Task3Option(100, 1.2, 1.2, 0.05, 0.03, 1, 0, 0.2, 0.1, 0.5)

np.float64(6.698252581126463)

# Monte Carlo Simulation

In [3]:
def Task3bOption(S, X, X0, r, r_f, T, t, volS, volX, rho, num_sim):
    w1 = np.random.standard_normal(num_sim)
    w2 = rho * w1 + np.sqrt(1 - rho**2) * np.random.standard_normal(num_sim)
    ST = S * np.exp((r_f - 0.5 * volS**2) * (T - t) + volS * np.sqrt(T - t) * w1)
    XT = X * np.exp((r - r_f - 0.5 * volX**2) * (T - t) + volX * np.sqrt(T - t) * w2)
    payoff = ST * np.maximum(XT - X0, 0)
    MC = np.exp(-r * (T - t)) * np.mean(payoff)
    MCstd = np.exp(-r * (T - t)) * np.std(payoff) / np.sqrt(num_sim)
    return MC, MCstd


MC, MCstd = Task3bOption(100, 1.2, 1.2, 0.05, 0.03, 1, 0, 0.2, 0.1, 0.5, 10000)
print(
    "The MC estimate for the option price is: ",
    MC - norm.ppf(0.975) * MCstd,
    " to ",
    MC + norm.ppf(0.975) * MCstd,
)

The MC estimate for the option price is:  6.53805196521525  to  6.932823288758224


It can be seen that the Monte Carlo estimate is very close to the analytical solution, confirming the correctness of our derivation.